In [1]:
from sentence_transformers import CrossEncoder
import pickle
import json
import os

model_name = './output/cross-encoder_hlf-rbtl3-2022-06-11'

In [2]:
## Load our cross-encoder. Use fast tokenizer to speed up the tokenization
model = CrossEncoder(model_name)

In [3]:
with open('results_for_cross.pkl', "rb") as f:
    results_for_cross = pickle.load(f)

In [4]:
with open('recall_results.json', 'r', encoding='utf-8') as f:
    recall_results = json.load(f)

In [8]:
%%time
rerank_results = {}
for question, passages, question_id, passages_ids in zip(results_for_cross.keys(), results_for_cross.values(), recall_results.keys(), recall_results.values()):
    model_inputs = [[question, passage[:256]] for passage in passages]
    scores = model.predict(model_inputs)

    #Sort the scores in decreasing order
    results = [{'score': score, 'id': passages_id} for score, passages_id in zip(scores, passages_ids)]
    results = sorted(results, key=lambda x: x['score'], reverse=True)
    
    results_ids = []
    for hit in results:
        results_ids.append(hit['id'])
    rerank_results[question_id] = results_ids

CPU times: user 54min 25s, sys: 1min 52s, total: 56min 18s
Wall time: 24min 43s


In [9]:
with open('rerank_results.json', 'w', encoding='utf-8') as f:
    json.dump(rerank_results, f, ensure_ascii=False, indent='\t')

In [10]:
len(rerank_results)

25000

In [ ]:
#Search in a loop for the individual queries
for query in queries:
    start_time = time.time()

    #Concatenate the query and all passages and predict the scores for the pairs [query, passage]
    model_inputs = [[query, passage] for passage in passages]
    scores = model.predict(model_inputs)

    #Sort the scores in decreasing order
    results = [{'input': inp, 'score': score} for inp, score in zip(model_inputs, scores)]
    results = sorted(results, key=lambda x: x['score'], reverse=True)

    print("Query:", query)
    print("Search took {:.2f} seconds".format(time.time() - start_time))
    for hit in results[0:5]:
        print("Score: {:.2f}".format(hit['score']), "\t", hit['input'][1])


    print("==========")